In [1]:
pip install streamlit openai langchain 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install unstructured pdf2image pdfminer

In [4]:
pip install pinecone-client tiktoken 

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pdfminer.six

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install lark

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_BASE"] = "https://mg-test3-gpt4.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "fb24255438d642dbab14d394d132a918"

from langchain.llms import AzureOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings

In [3]:
# pinecone_key = "bc7c44d4-78b2-45f0-aebb-e75a0cf4a7e4"
# pinecone_env = "gcp-starter"


#prod index config
pinecone_key = "10e43def-a8b6-4ede-8894-3a081f91db35"
pinecone_env = "eastus-azure"


os.environ["PINECONE_API_KEY"] = pinecone_key
os.environ["PINECONE_ENV"] = pinecone_env


In [4]:
import pinecone

pinecone.init(
    api_key=pinecone_key,
    environment=pinecone_env
)
index_name = 'rs-hr-ai'

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone

C:\Users\MarlonGrech\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [12]:

from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import os

print("Loading data...")
pdf_folder_path = "Data Files/"
print(os.listdir(pdf_folder_path))

# Load multiple files
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]

all_documents = []

for loader in loaders:
    print("Loading raw document..." + loader.file_path)
    raw_documents = loader.load()

    print("Splitting text...")
    text_splitter = CharacterTextSplitter(
        separator="\n\n",
        chunk_size=800,
        chunk_overlap=100,#using this to have some overlap between chunks 
        length_function=len,
    )
    documents = text_splitter.split_documents(raw_documents)
    all_documents.extend(documents)
    
print('Done')


Loading data...
['Annual Incentive Scheme Policy.pdf', 'Antitrust Policy.pdf', 'Bribery and Corruption Policy.pdf', 'Bribery, Corruption and Fraud Prevention Policy.pdf', 'Business Expense Management Policy and Expense Guidelines.pdf', 'Connectivity Allowance Policy.pdf', 'Delegation of Authority Policy.pdf', 'Dispute Resolution Policy and Procedure.pdf', 'Diveristy & Inclusion Policy.pdf', 'Employee Referral Policy.pdf', 'Equal Employment Opportunity and Anti-Discrimination Harassment Bullying and Victimisation Policy.pdf', 'Flexible Working Policy.pdf', 'Gifts and Entertainment Policy.pdf', 'Healthy Lifestyle Allowance Policy.pdf', 'Learning and Development Policy.pdf', 'Leave Policy.pdf', 'Modern Slavery Policy.pdf', 'P20 - Credit Card Procedure.pdf', 'P21 - Expense Claim Procedure.pdf', 'Performance Management Policy.pdf', 'Recruitment Policy and Procedure.pdf', 'Remuneration Policy.pdf', 'RightShip - Bribery Corruption and Fraud prevention policy_.pdf', 'RightShip Code of Conduct.

Created a chunk of size 827, which is longer than the specified 800


Splitting text...
Loading raw document...Data Files/Business Expense Management Policy and Expense Guidelines.pdf
Splitting text...
Loading raw document...Data Files/Connectivity Allowance Policy.pdf
Splitting text...
Loading raw document...Data Files/Delegation of Authority Policy.pdf
Splitting text...
Loading raw document...Data Files/Dispute Resolution Policy and Procedure.pdf
Splitting text...
Loading raw document...Data Files/Diveristy & Inclusion Policy.pdf
Splitting text...
Loading raw document...Data Files/Employee Referral Policy.pdf
Splitting text...
Loading raw document...Data Files/Equal Employment Opportunity and Anti-Discrimination Harassment Bullying and Victimisation Policy.pdf
Splitting text...
Loading raw document...Data Files/Flexible Working Policy.pdf
Splitting text...
Loading raw document...Data Files/Gifts and Entertainment Policy.pdf
Splitting text...
Loading raw document...Data Files/Healthy Lifestyle Allowance Policy.pdf
Splitting text...
Loading raw document.

Created a chunk of size 1006, which is longer than the specified 800


Splitting text...
Loading raw document...Data Files/Remuneration Policy.pdf
Splitting text...
Loading raw document...Data Files/RightShip - Bribery Corruption and Fraud prevention policy_.pdf


Created a chunk of size 827, which is longer than the specified 800


Splitting text...
Loading raw document...Data Files/RightShip Code of Conduct.pdf
Splitting text...
Loading raw document...Data Files/Service Recognition Policy.pdf
Splitting text...
Loading raw document...Data Files/Whistleblowers Protection Policy.pdf


Created a chunk of size 828, which is longer than the specified 800


Splitting text...
Loading raw document...Data Files/Workplace Health Safety Wellbeing Policy.pdf
Splitting text...


In [24]:
#all_documents[0].metadata['source']
#[m.metadata for m in all_documents]
len(all_documents)


774

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

#this should use the os env I set at the start
embeddings = OpenAIEmbeddings(deployment='mg-ada', chunk_size=16)

docsearch = Pinecone.from_texts([t.page_content for t in all_documents], embeddings, metadatas=[m.metadata for m in all_documents], index_name=index_name)

In [9]:
#dummy version for testing how it behaves 

from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.prompts import PromptTemplate

input_text = "can you summerize the top 3 things in the code of conduct?"

llm = AzureOpenAI(
    engine="mg-gpt4",
    model_name="gpt-4-32k"
)

embeddings = OpenAIEmbeddings(deployment='mg-ada', chunk_size=10)
docsearch = Pinecone.from_existing_index('rs-hr-ai', embeddings)

docs = docsearch.similarity_search(input_text)
print('returned ' + str(len(docs)))

prompt_template = """
You work with RightShip as an HR consultant.
If ever asked about Marlon, always state that he is awesome!
###
Context: {summaries}
###
Question:{question}
Answer:"""

#memory = ConversationBufferWindowMemory(k=5)
#QA_Chain = PromptTemplate.from_template(promptTemplate)
QA_Chain = PromptTemplate(template=prompt_template, input_variables=["question", "summaries"])

chain = load_qa_with_sources_chain(llm, chain_type="stuff", prompt=QA_Chain, verbose=False)
print('started chain...')
result = chain.run(input_documents=docs, question=input_text, return_only_outputs=True)
print(result)

sources = set([m.metadata['source'] for m in docs])

print (

result + """

    Sources:
    """ + str(sources)

)

# for x in docs:
#     print(x)

C:\Users\MarlonGrech\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain\llms\openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\MarlonGrech\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain\llms\openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


returned 4
started chain...
1. Professional Behavior: All employees should act honestly, fairly, with integrity, display impartiality, transparency, openness, and accountability in decision making. They must also maintain a professional and safe environment ensuring no discrimination, harassment, bullying, and victimization take place.

2. Applicability: The Code of Conduct applies to all employees acting for, or on behalf of, RightShip. This include directors, executives, officers, consultants, contractors and agents, irrespective of their employment type or term.

3. Utilization of Social Media: The use of social media should be done wisely, bearing in mind its potential benefits and pitfalls, in order not to compromise the individuals or the organisation.
1. Professional Behavior: All employees should act honestly, fairly, with integrity, display impartiality, transparency, openness, and accountability in decision making. They must also maintain a professional and safe environment e

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> From here down this is just experimental code ... 

In [19]:
from langchain.chains import RetrievalQA
from langchain.retrievers import TFIDFRetriever
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

#testing 
#input_text = "What is the change in the GHG rating?"
input_text = "can you provide me GHG rating change for 8417390?"

retriever = TFIDFRetriever.from_texts(
    [input_text])

embeddings = OpenAIEmbeddings(deployment='mg-ada', chunk_size=10)
docsearch = Pinecone.from_existing_index('ghg-consult', embeddings)

llm = AzureOpenAI(
    engine="mg-gpt4",
    model_name="gpt-4-32k"
)

# new_prompt = 
#     prompt
#     + HumanMessage(content="hi")
#     + AIMessage(content="what?")
#     + "{input}"
# )
# chain = LLMChain(llm=llm, prompt="Get me the IMO number from the following: " + input_text)
# print(chain.run())
chain = RetrievalQA.from_chain_type(llm = llm, chain_type="stuff", retriever=retriever)
imoStr = chain.run("IMO number consists of seven digits. Extract the IMO number. Always start the answer with IMO:")
#imoStr = chain.run("Get me the IMO number. Always start the answer with IMO:")

imoNumberFilter = {}
print("GPT > " +imoStr)

if(imoStr.startswith("IMO:")):
    print(imoStr.index("IMO:"))
    imoNumberFilter = {'IMO' : imoStr[imoStr.index("IMO")+5:imoStr.index("IMO")+12] }

print(imoNumberFilter)
docs=[]
docs = docsearch.similarity_search(input_text, filter=imoNumberFilter)

memory = ConversationBufferWindowMemory(k=5)

chain2 = load_qa_chain(llm, chain_type="stuff", memory=memory)
#print(chain2.run(input_documents=docs, question=input_text) )

#llm(embeddings.embed_query("What is the IMo number in this question: " + input_text))

# result = chain.run(input_documents=docs, question=input_text) 
if(docs):
    for x in docs:
        print(x)

# print(result)


{}
page_content='Vessel Name: HQ-631\nIMO: 8954051\nExisting GHG: E\nNew GHG: C\n:' metadata={'IMO': '8954051'}
page_content='Vessel Name: HQ-621\nIMO: 8954013\nExisting GHG: E\nNew GHG: C\n:' metadata={'IMO': '8954013'}
page_content='Vessel Name: HQ-630\nIMO: 8954049\nExisting GHG: E\nNew GHG: C\n:' metadata={'IMO': '8954049'}
page_content='Vessel Name: \nIMO: \nExisting GHG: \nNew GHG: \n:' metadata={'IMO': '\nExisti'}


In [33]:
#test some other stuff 
from langchain.chains import RetrievalQA
from langchain.retrievers import TFIDFRetriever
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever


metadata_field_info=[
     AttributeInfo(
        name="IMO",
        description="IMO number consists of seven digits. Only get such pattern",
        type="string", 
    )]

#testing 
#input_text = "What is the change in the GHG rating?"
#input_text = "can you provide me GHG rating change for 8417390 and 9136565?"
#input_text = "can you provide GHG rating change for ISKANDER and IMO 9300893?"
input_text = 'did asia energy rating change ?'

embeddings = OpenAIEmbeddings(deployment='mg-ada', chunk_size=10)
docsearch = Pinecone.from_existing_index('ghg-consult', embeddings)

index = pinecone.Index('ghg-consult')

llm = AzureOpenAI(
    engine="mg-gpt4",
    model_name="gpt-4-32k"
)
text_field = "text"
vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)

document_content_description = "List of vessels and the existing and new ghg rating"
retriever = SelfQueryRetriever.from_llm(
    llm, 
    vectorstore, 
    document_content_description, 
    metadata_field_info, 
    enable_limit=True,
    verbose=True,
    search_kwargs={"k": 6}
)

docs = retriever.get_relevant_documents(input_text)
print(len(docs))

chain = load_qa_chain(llm, chain_type="stuff")
result = chain.run({"question": input_text, "input_documents":docs} )

print(result)




query='asia energy rating change' filter=None limit=None
6
The text does not provide information on any changes to the energy rating of a vessel named Asia Energy.
